In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
 
import numpy as np 
import pandas as pd 
 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import itertools
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import sklearn.tree as tree
#from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/content/forestfires.csv')
df.head()
df=df.drop(columns=['X','Y','day'])


FileNotFoundError: ignored

In [ ]:
def damage_func(area):
    if area == 0.0:
        return 0
    else:
        return 1

df['damage'] = df['area'].apply(damage_func)

In [ ]:
df1 = pd.read_csv('/content/Algerian_forest_fires_dataset_UPDATE.csv')

In [ ]:
print(df.columns,"/n",df1.columns)

In [ ]:
month_dict = {'mar': 3, 'oct': 10, 'aug': 8, 'sep': 9, 'apr': 4, 'jun': 6, 'jul': 7,
              'feb': 2, 'jan': 1, 'dec': 12, 'may': 5, 'nov': 11}
day_dict = {'fri': 5, 'tue': 2, 'sat': 6, 'sun': 7, 'mon': 1, 'wed': 3, 'thu': 4}


In [ ]:
df.replace(month_dict.keys(),month_dict.values(),inplace=True)
df.replace(day_dict.keys(),day_dict.values(),inplace=True)
df.head()

In [ ]:
new_column_names = {'Temperature': 'temp', 'Rain ': 'rain','Classes  ':'damage',' RH':'RH',' Ws':'wind'}
df1 = df1.rename(columns=new_column_names)

In [ ]:
df1.head()

In [ ]:
df1['damage'][0]

In [ ]:
def damage_func1(damage):
    if damage == 'not fire   ':
        return 0
    else:
        return 1

df1['damage'] = df1['damage'].apply(damage_func1)

In [ ]:
df1.head()

In [ ]:
df.info()

In [ ]:
df1=df1.drop(columns=['day','BUI','FWI','year'])
display(df.dtypes)
display(df1.dtypes)


In [ ]:
df1.isna().sum()
df1=df1.dropna()
df1.isna().sum()

In [ ]:
# Replace non-numeric values with NaN
df1['month'] = pd.to_numeric(df1['month'], errors='coerce')

# Drop rows with missing values in 'month' column
df1 = df1.dropna(subset=['month'])


In [ ]:
df1_copy = df1.copy()
df1_copy.loc[:, 'month'] = df1_copy['month'].astype('int64')
df1_copy.loc[:, 'temp'] = df1_copy['temp'].astype('float64')
df1_copy.loc[:, 'RH'] = df1_copy['RH'].astype('int64')
df1_copy.loc[:, 'wind'] = df1_copy['wind'].astype('float64')
df1_copy.loc[:, 'rain'] = df1_copy['rain'].astype('float64')
df1_copy.loc[:, 'FFMC'] = df1_copy['FFMC'].astype('float64')
df1_copy.loc[:, 'DMC'] = df1_copy['DMC'].astype('float64')
df1_copy.loc[:, 'DC'] = pd.to_numeric(df1_copy['DC'], errors='coerce').astype('float64')
df1_copy.loc[:, 'ISI'] = pd.to_numeric(df1_copy['ISI'], errors='coerce').astype('float64')
df1_copy=df1_copy.dropna()

In [ ]:
display(df1_copy.dtypes)

In [ ]:
df = pd.concat([df, df1_copy], ignore_index=True)

In [ ]:
df

# Features

In [ ]:
df=df.drop(columns='area')
df.head()

In [ ]:
# # Categorical: Month and day
# df = pd.get_dummies(df,columns=['day','month'],drop_first=True)

# Model Building

After model fitting, I apply the reverse log transform, to compare with test data (RMSE has same units as target, and it's more intuitive to report in units of area rather than log area).

In [ ]:

df=df.dropna()
# Creating bins of in the target, to stratify in train/test split
df['damage_bin'] = pd.cut(df['damage'], 5)
X = df.drop(columns='damage') 
y = df['damage']


First, there is a bit of an issue due to the dataset's small size. I noticed setting a random seed for reproducibility had a large affect on the final model and predictions. I illustrate this below:

In [ ]:
rs = 99
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=rs,
                                                   stratify=df[['damage_bin']])
X_train = X_train.drop(columns='damage_bin',axis=1)
X_test = X_test.drop(columns='damage_bin',axis=1)

# Baseline Models

**Random Forest**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestRegressor()
rf=RandomForestClassifier()
rf.fit(X_train, Y_train)
pipe1 = make_pipeline(
    StandardScaler(),
    RandomForestClassifier()
)
pipe1.fit(X_train, Y_train)

y_pred1 = pipe1.predict(X_test)
y_pred = rf.predict(X_test)

print(accuracy_score(Y_test, y_pred1))
print(accuracy_score(Y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, Y_train)
y_pred = knn_classifier.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, Y_train)
y_pred = logreg_classifier.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print('Accuracy:', accuracy)



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
cm = confusion_matrix(Y_test, y_pred)
class_labels = ['No damage', 'damage']
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
pipe = make_pipeline(
    StandardScaler(),
    LogisticRegression()
)
pipe.fit(X_train, Y_train)
y_pred = pipe.predict(X_test)


In [ ]:
accuracy = accuracy_score(Y_test, y_pred)

In [ ]:
accuracy

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a Decision Tree Classifier object
clf = DecisionTreeClassifier(criterion='gini', splitter='best')
clf.fit(X_train, Y_train)
accuracy = accuracy_score(Y_test, y_pred)
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create a Random Forest Classifier object
rf_classifier = RandomForestClassifier(n_estimators=100)

# Create an AdaBoost Classifier object with the Random Forest Classifier as the base estimator
adaboost_classifier = AdaBoostClassifier(base_estimator=rf_classifier, n_estimators=50)


# Train the AdaBoost classifier on the training data
adaboost_classifier.fit(X_train, Y_train)

# Make predictions on the test data
y_pred = adaboost_classifier.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
